In [1]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
base_url = "https://demo.hafas.de/openapi/vbb-proxy/"

In [3]:
ACCESS_ID = "pdyban-4097-9a8e-f21a391aac8f"

In [4]:
def req(endpoint):
    response = requests.get(endpoint)
    js = json.loads(response.text)
    return js

In [5]:
origin = (52.5159984,13.4575739)  # Frankfurter Allee 15
dest = (52.5219216,13.411026)  # Alexanderplatz

In [6]:
def append_access_id(endpoint):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "accessId={ACCESS_ID}".format(ACCESS_ID=ACCESS_ID)

def force_json_format(endpoint):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "format=json"

def set_latlon(endpoint, point, mode):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + \
            "{mode}CoordLat={point[0]}".format(mode=mode, point=point) + '&' + \
            "{mode}CoordLon={point[1]}".format(mode=mode, point=point)

def set_origin_latlon(endpoint, point):
    return set_latlon(endpoint, point, 'origin')

def set_dest_latlon(endpoint, point):
    return set_latlon(endpoint, point, 'dest')

def set_extid(endpoint, extid, mode):
    return endpoint + ('?','&')[endpoint.find('?') > -1] + "{}ExtId={}".format(mode,extid)

def set_origin_extid(endpoint, extid):
    return set_extid(endpoint, extid, 'origin')

def set_dest_extid(endpoint, extid):
    return set_extid(endpoint, extid, 'dest')

Resolve origin and dest HAFAS locations using nearbystops

In [7]:
def get_closest_stop(point):
    endpoint = base_url + 'location.nearbystops'
    endpoint = append_access_id(endpoint)
    endpoint = force_json_format(endpoint)
    endpoint += '&originCoordLong={}&originCoordLat={}&maxNo=1&r=1000'.format(origin[1],origin[0])
    response = req(endpoint)
    for key, result in response.items():
        for location in result:
            # print(location['StopLocation'], location['StopLocation']['extId'])
            return location['StopLocation']
        
    return None

In [8]:
origin_ext = get_closest_stop(origin)
origin_extid = origin_ext['extId']
dest_extid = '900100003'

In [9]:
pt = (52.52448133333333, 13.46405325)
endpoint = base_url + 'location.nearbystops'
endpoint = append_access_id(endpoint)
endpoint = force_json_format(endpoint)
endpoint += '&originCoordLong={}&originCoordLat={}&maxNo=1&r=1000'.format(pt[1],pt[0])
response = req(endpoint)
print(response)

{'stopLocationOrCoordLocation': [{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'name': 'S8', 'line': 'S8', 'lineId': 'S8', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-8', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S41', 'line': 'S41', 'lineId': 'S41', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-3', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S42', 'line': 'S42', 'lineId': 'S42', 'catOut': 'S', 'cls': '1', 'catOutS': 'S-3', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': 'S85', 'line': 'S85', 'lineId': 'S85', 'catOut': 'S', 'cls': '1', 'catOutS': 'S12', 'catOutL': 'S'}, {'icon': {'res': 'prod_gen'}, 'name': '156', 'line': '156', 'lineId': '156', 'catOut': 'Bus', 'cls': '8', 'catOutS': 'B', 'catOutL': 'Bus'}, {'icon': {'res': 'prod_gen'}, 'name': '240', 'line': '240', 'lineId': '240', 'catOut': 'Bus', 'cls': '8', 'catOutS': 'B', 'catOutL': 'Bus'}], 'id': 'A=1@O=S Storkower Str. (Berlin)@X=13464834@Y=52524052@u=0@U=86@L=900110012@', 'extId': '

Request all trips between origin and dest

In [10]:
def total_travel_time(response):
    ttt = []
    for trip in response['Trip']:
        leglist = trip['LegList']

        for key in leglist:
            travel_time = timedelta()
            for leg in leglist['Leg']:
                # print(leg, len(leg))
                leg_origin, leg_dest = leg['Origin'], leg['Destination']
                # print(leg_origin['time'], leg_dest['time'])
                leg_origin_time = datetime.strptime(leg_origin['time'], '%H:%M:%S')
                leg_dest_time = datetime.strptime(leg_dest['time'], '%H:%M:%S')
                td = timedelta() + leg_dest_time - leg_origin_time
                travel_time += td
            # print('TOTAL TRAVEL TIME: {}'.format(travel_time))
            ttt.append(travel_time)
    return ttt

def compute_travel_times(origin_extid, dest_extid):
    endpoint = base_url + 'trip'
    endpoint = append_access_id(endpoint)
    endpoint = force_json_format(endpoint)
    endpoint = set_origin_extid(endpoint, origin_extid)
    endpoint = set_dest_extid(endpoint, dest_extid)
    return total_travel_time(req(endpoint))

In [11]:
compute_travel_times(origin_extid, dest_extid)

[datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=360),
 datetime.timedelta(seconds=360)]

Consider distance from the requested GPS coordinate to the nearest stop

In [12]:
origin = (52.517602, 13.458108)

In [35]:
origin_closest_stop = get_closest_stop(origin)
origin_extid = origin_closest_stop['extId']
origin_closest_stop_coord = (origin_closest_stop['lat'], origin_closest_stop['lon'])

In [36]:
import math

def distance_utf(point1, point2):
    def toRadians(deg):
        return deg * math.pi / 180.0
    
    R = 6371e3; # metres
    lat1, lon1 = point1
    lat2, lon2 = point2
    phi1 = toRadians(lat1)
    phi2 = toRadians(lat2)
    delta_phi = toRadians(lat2-lat1)
    delta_lambda = toRadians(lon2-lon1)

    a = math.sin(delta_phi/2) * math.sin(delta_phi/2) + \
            math.cos(phi1) * math.cos(phi2) * \
            math.sin(delta_lambda/2) * math.sin(delta_lambda/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    d = R * c
    return d

In [37]:
travel_times = compute_travel_times(origin_extid, dest_extid)
travel_times

[datetime.timedelta(seconds=540),
 datetime.timedelta(seconds=480),
 datetime.timedelta(seconds=900),
 datetime.timedelta(seconds=480),
 datetime.timedelta(seconds=900),
 datetime.timedelta(seconds=540),
 datetime.timedelta(seconds=480)]

In [38]:
distance_utf(origin, origin_closest_stop_coord)

284.97535065497823

In [39]:
WALK_VELOCITY = 3000/3600.0 # m/s
distance_to_closest_stop = distance_utf(origin, origin_closest_stop_coord)
travel_times_adjusted = []
for tt in travel_times:
    travel_times_adjusted.append(tt + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY))
travel_times_adjusted

[datetime.timedelta(seconds=881, microseconds=970421),
 datetime.timedelta(seconds=821, microseconds=970421),
 datetime.timedelta(seconds=1241, microseconds=970421),
 datetime.timedelta(seconds=821, microseconds=970421),
 datetime.timedelta(seconds=1241, microseconds=970421),
 datetime.timedelta(seconds=881, microseconds=970421),
 datetime.timedelta(seconds=821, microseconds=970421)]

Record data into a CSV file for later rendering

In [40]:
df = pd.DataFrame(columns=['latitude', 'longitude', 'min travel time'])

origins = [
    (52.5159984,13.4575739),
    (52.517602, 13.458108)
]
WALK_VELOCITY = 3000/3600.0 # m/s

for origin in origins:
    origin_closest_stop = get_closest_stop(origin)
    distance_to_closest_stop = distance_utf(origin, origin_closest_stop_coord)
    origin_extid = origin_closest_stop['extId']
    origin_closest_stop_point = (origin_closest_stop['lat'], origin_closest_stop['lon'])
    
    travel_times = compute_travel_times(origin_extid, dest_extid)

    travel_times_adjusted = []
    for tt in travel_times:
        travel_times_adjusted.append(tt + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY))

    df = df.append({'latitude': origin[0], 'longitude': origin[1], 'min travel time': min(    travel_times_adjusted).total_seconds()}, ignore_index=True)

In [41]:
df

,latitude,longitude,min travel time
0,52.515998,13.457574,833.557670
1,52.517602,13.458108,821.970421


In [42]:
df.to_csv('hafas_vbb.csv')

# Compute travel time for an equally spaced rectangular area

In [47]:
topleft = (52.526453, 13.447507)
bottomright = (52.502793, 13.473981)
num_points = 5
WALK_VELOCITY = 3.0/3.6

df = pd.DataFrame(columns=['latitude', 'longitude', 'min travel time'])

origins = []
lat_arr = np.linspace(bottomright[0], topleft[0], num_points)
lon_arr = np.linspace(bottomright[1], topleft[1], num_points)
for lat in lat_arr:
    for lon in lon_arr:
        origins.append((lat, lon))

for origin in origins:
    origin_closest_stop = get_closest_stop(origin)
    origin_extid = origin_closest_stop['extId']
    origin_closest_stop_point = (origin_closest_stop['lat'], origin_closest_stop['lon'])
    distance_to_closest_stop = distance_utf(origin, origin_closest_stop_point)
    
    travel_times = compute_travel_times(origin_extid, dest_extid)

    travel_times_adjusted = []
    for tt in travel_times:
        travel_times_adjusted.append(tt + timedelta(seconds=distance_to_closest_stop/WALK_VELOCITY))

    df = df.append({'latitude': origin[0], 'longitude': origin[1], 'min travel time': min(travel_times_adjusted).total_seconds()}, ignore_index=True)
    print('.', end='')

.........................

In [50]:
df.head()

,latitude,longitude,min travel time
0,52.502793,13.473981,904.584445
1,52.502793,13.467363,636.986839
2,52.502793,13.460744,859.758908
3,52.502793,13.454125,846.968438
4,52.502793,13.447507,499.271294


In [49]:
df.to_csv('hafas_vbb.csv')